In [2]:
import pandas as pd
import numpy as np
from symbol import Symbol
from index import Index
from portifolio import Portifolio
from datetime import datetime
import os
import matplotlib.pyplot as plt
from bqplot import pyplot as bplt
import seaborn as sns
from dateutil.relativedelta import relativedelta

sns.set(context='notebook', style="dark", palette="muted", color_codes=True)

%matplotlib inline

In [2]:
s= Symbol()
i = Index()
p = Portifolio()

In [3]:
s.symbol_meta.head()

,name_of_company,date_of_listing,from_date,to_date,row_count,industry,mcap,div_yield,p_e_ratio,bse_identifier,isin_identifier,promoter_holding,fii_holding,dii_mf_holding,others_holding
symbol,,,,,,,,,,,,,,,
20microns,20 Microns Limited,2008-10-06,2008-10-06,2017-01-11,2046,metals,142.0,0.0,13.690,533022.0,INE144J01027,50.0,0.0,0.0,50.0
3iinfotech,3i Infotech Limited,2005-04-22,2005-04-22,2017-01-11,2644,it,623.0,0.0,-0.544,532628.0,INE748C01020,3.0,4.0,34.0,59.0
3mindia,3M India Limited,2004-08-13,2004-08-13,2017-01-11,3080,trading,13855.0,0.0,65.500,523395.0,INE470A01017,75.0,9.0,3.0,13.0
8kmiles,8K Miles Software Services Limited,2014-01-29,2014-01-29,2017-01-11,726,it,1942.0,0.0,23.590,512161.0,INE650K01021,63.0,0.0,2.0,34.0
a2zinfra,A2Z INFRA ENGINEERING LIMITED,2010-12-23,2010-12-23,2017-01-11,1381,power,561.0,0.0,NaN,533292.0,INE619I01012,46.0,2.0,6.0,46.0


In [4]:
ret = s.get_symbol_data()
close = s.get_symbol_data('close')
volume = s.get_symbol_data('volume')
ret = ret['2016']
close = close['2016']
volume = volume['2016']
iret = i.get_index_returns('nifty_50')
iclose = i.get_index_close(index_list='nifty_50')
iret = iret['2016']
iclose = iclose['2016']

In [5]:
from helpers import num_missing
null_count = ret.apply(num_missing, start='beginning')
symbol_list = null_count[null_count == 0].index
symbol_list = list(symbol_list)

In [6]:
ret = ret[symbol_list]
close = close[symbol_list]
volume = volume[symbol_list]
adj_close = pd.DataFrame(p.prices(ret))
adj_close = (close/close.ix[-1, :])*close.ix[-1].round(4)

In [7]:
def high_low(adj_close):
    des = pd.DataFrame(index=adj_close.columns)
    des['low'] = adj_close.min()
    des['high'] = adj_close.max()
    des['mean'] = adj_close.max()
    des['median'] = adj_close.median()
    
    return des
close_stats = high_low(adj_close)
volume_stats = high_low(volume).astype(int)
close_stats.head()

,low,high,mean,median
symbol,,,,
20microns,25.45,43.15,43.15,32.25
3iinfotech,3.80,6.80,6.80,5.05
3mindia,9521.50,14939.55,14939.55,12638.25
8kmiles,591.30,2483.70,2483.70,1868.55
a2zinfra,17.40,49.05,49.05,36.05


In [8]:
def returns_stats(ret, iret):
    iret = iret.nifty_50
    des = pd.DataFrame(index=ret.columns)
    des['sum_returns'] = ret.sum()
    des['min_returns'] = ret.min()
    des['high_returns'] = ret.max()
    des['mean_returns'] = ret.mean()
    des['median_returns'] = ret.median()
    
    risk_free_rate, _ = s.set_risk_free_rate(ret)
    des['hpr'] = np.exp(ret.sum())*100 - 100
    des['hpr_rf'] = np.exp(ret.sum() - risk_free_rate.sum())*100 - 100
    des['hpr_bench'] = np.exp(ret.sum() - iret.sum())*100 - 100

    des['std_dev'] = ret.std()
    des['lpm'] = p.lower_partial_moment(ret)
    des['hpm'] = p.higher_partial_moments(ret)
    des['pos_pctile'] = ret[ret>0].count() / ret.count()
    
    
    
    return des
ret_stats = returns_stats(ret, iret)
ret_stats.head()

,sum_returns,min_returns,high_returns,mean_returns,median_returns,hpr,hpr_rf,hpr_bench,std_dev,lpm,hpm,pos_pctile
symbol,,,,,,,,,,,,
20microns,-0.102321,-0.098218,0.178966,-0.000414,0.000000,-9.726027,-15.575530,-12.366710,0.029477,0.019607,0.021934,0.465587
3iinfotech,0.152677,-0.106295,0.163629,0.000618,0.000000,16.494845,8.946303,13.087153,0.034119,0.021215,0.026641,0.368421
3mindia,-0.024374,-0.082843,0.153247,-0.000099,-0.000478,-2.407974,-8.731666,-5.262723,0.020390,0.012013,0.016425,0.469636
8kmiles,-0.287392,-0.214198,0.174936,-0.001164,-0.003109,-24.978228,-29.839430,-27.172755,0.036794,0.025412,0.026531,0.408907
a2zinfra,0.440899,-0.163432,0.180710,0.001785,-0.001315,55.410448,45.340282,50.864401,0.041948,0.026173,0.032721,0.477733


In [9]:
capm_prices = p.calculate_capm(close, iclose).set_index('symbol').drop(['benchmark'], axis=1)
capm_prices.head()

,alpha,beta,std_dev,r_square
symbol,,,,
20microns,-15.519320,0.005942,4.458844,0.450033
3iinfotech,-2.836240,0.000970,0.735341,0.440818
3mindia,-3019.230725,1.874128,1294.924976,0.530793
8kmiles,3249.604852,-0.198135,542.858917,0.033757
a2zinfra,-82.763837,0.014238,8.577377,0.698279


In [10]:
def returns_ratios(returns, iret):
    returns_describe = p.calculate_capm(returns, iret).set_index('symbol').drop(['benchmark'], axis=1)
    sharpe_ratio = p.sharpe_ratio(returns)
    returns_describe = returns_describe.join(sharpe_ratio)
    sortino_ratio = p.sortino_ratio(returns, 0)
    returns_describe = returns_describe.join(sortino_ratio)
    upside_potential_ratio = p.upside_potential_ratio(returns, 0)
    returns_describe = returns_describe.join(upside_potential_ratio)
    return returns_describe
ratios_returns = returns_ratios(ret, iret)
ratios_returns.head()

,alpha,beta,std_dev,r_square,sharpe_ratio,sortino_ratio,upside_potential_ratio
symbol,,,,,,,
20microns,-0.000519,1.104112,0.029478,0.127880,-0.366208,-0.550556,1.118674
3iinfotech,0.000506,1.053774,0.034120,0.086944,0.160113,0.257514,1.255779
3mindia,-0.000260,0.729387,0.020389,0.116645,-0.285700,-0.484908,1.367219
8kmiles,-0.001161,1.813710,0.036794,0.221481,-0.614077,-0.889115,1.044015
a2zinfra,0.001834,2.119675,0.041949,0.232732,0.568293,0.910831,1.250189


In [11]:
def min_max_correlation(returns):
    corr_mat = pd.DataFrame(index=returns.columns)
    correlation = returns.corr()
    correlation = correlation.replace(1, np.nan)
    corr_mat['min_corr'] = correlation.min()
    corr_mat['min_corr_symbol'] = correlation.idxmin()
    corr_mat['max_c0rr'] = correlation.max()
    corr_mat['max_corr_symbol'] = correlation.idxmax()
    return corr_mat
min_max_corr = min_max_correlation(ret)
min_max_corr.head()

,min_corr,min_corr_symbol,max_c0rr,max_corr_symbol
symbol,,,,
20microns,0.017390,cyient,0.482799,bbtc
3iinfotech,-0.010682,kirlosind,0.465779,jswenergy
3mindia,-0.039266,tokyoplast,0.475518,insecticid
8kmiles,0.003430,mangtimber,0.496592,prismcem
a2zinfra,-0.006183,swanenergy,0.577037,gmrinfra


In [12]:
from market import INDEX_META
INDEX_META.index
smsa = pd.DataFrame(index=s.symbol_list.index.copy())
for index_type in INDEX_META.category.unique():
    data = s.get_symbol_index(index_type)
    for index in data.columns:
        data[index] = data[index].replace(True, index).replace(False, np.nan)
    data = data.apply(lambda x: list(x.dropna().values), axis=1).rename(index_type)
    smsa = smsa.join(data)
smsa.head()

,mkt_index,stg_index,sec_index,thematic_index
symbol,,,,
20microns,[],[],[],[]
3iinfotech,[],[],[],[]
3mindia,"[nifty_midcap_150, nifty_free_float_midsmallca...",[],[],[]
8kmiles,"[nifty_smallcap_250, nifty_free_float_midsmall...",[],[],[]
a2zinfra,[],[],[],[]


In [13]:
# store = pd.HDFStore('symbol_data.h5', 'r')
# store.close()

In [14]:
data = s.get_symbol_hist('infy').set_index('date')['2016']

In [15]:
EMA(data, 20)[['close', 'EMA_20']].plot()

NameError: name 'EMA' is not defined

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=[30,10], )
left = sns.regplot(x=iclose.nifty_50, y=close['tatamotors'], ax=axes[0])
right = sns.regplot(x=iret.nifty_50, y=ret['tatamotors'], ax=axes[1])
axes[0].set_title('Close Prices', fontsize=24)
axes[1].set_title('Daily Returns', fontsize=24)

In [ ]:
stg_index = pd.read_hdf('symbol_data.h5', 'stg_index')
sec_index = pd.read_hdf('symbol_data.h5', 'sec_index')

In [ ]:
index_symbol = pd.concat([stg_index, sec_index], axis=1, keys=['stg_index', 'sec_index'])

In [ ]:
index_symbol[['sec_index', ]]

In [ ]:
index_symbol.loc[:, 'stg_index']

In [ ]:
slice(1, 5)

In [ ]:
1:5